In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import csv
import cv2
import tqdm.notebook as tq
from sklearn.utils import shuffle
from keras.optimizers import *
from keras.losses import *
from keras import Sequential, Model
from keras.layers import *
import PIL
import PIL.Image
import keras.backend as K
#tf.config.run_functions_eagerly(True)

In [4]:
data_location = 'Dataset/'
model_location = 'Models/'

In [5]:
def readTrafficSigns(rootpath, num_classes):
    '''Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = []
    labels = []
    for c in tq.tqdm( range(0,43) ):
        prefix = rootpath + '/' + format(c, '05d') + '/' 
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') 
        gtReader = csv.reader(gtFile, delimiter=';') 
        next(gtReader) 
        
        for row in gtReader:
            img = plt.imread(prefix + row[0])
            img = cv2.resize(img, (30, 30))
            d = np.zeros(num_classes)
            d[int(row[7])] = 1
            images.append(img) 
            labels.append(d)
        gtFile.close()
    return np.array(images), np.array(labels)

In [6]:
def get_discriminator(in_shape):
    
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.00001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def get_generator(in_shape):
    
    n_nodes      = 128 * 15 * 15
    input_image  = Input(in_shape)
    #rescaled    = experimental.preprocessing.Rescaling(1./255)(input_image)
    flat         = Flatten() (input_image)
    fc1          = Dense(n_nodes, kernel_initializer=tf.keras.initializers.HeNormal()) (flat)
    lr1          = ReLU() (fc1)
    out1         = Reshape((15, 15, 128)) (lr1)
    tconv1       = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
                                  kernel_initializer=tf.keras.initializers.HeNormal()) (out1)
    lr2          = LeakyReLU(alpha=0.2) (tconv1)
    noise        = Conv2D(3, (7,7), activation='sigmoid', padding='same',
                          kernel_initializer=tf.keras.initializers.HeNormal()) (lr2)
    
    #noise = tf.clip_by_value(noise, clip_value_min=-0.15, clip_value_max=0.15)
    output_image = 0.9 * input_image +  0.1 *noise 
    output_image = tf.clip_by_value(output_image, clip_value_min=0, clip_value_max=1)
    
    model = Model(inputs=input_image, outputs=output_image)
    opt = Adam(lr=0.001, beta_1=0.6)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def define_gan(generator, discriminator):   # Discriminator Loss Model
    
    discriminator.trainable = False
    model = Sequential()
    model.add( generator )
    model.add( discriminator )
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def custom_loss(y_actual, y_predicted):
    loss = K.categorical_crossentropy(y_actual, y_predicted)
    return -1 * loss

def define_adv(generator, classifier):
    classifier.trainable = False
    model = Sequential()
    model.add( generator )
    model.add( classifier )
    opt = Adam(lr=0.004, beta_1=0.5)
    model.compile(loss=custom_loss, optimizer='adam')
    return model

In [7]:
def generate_real_datas(train_ds):
    image_batch, label_batch = next(iter(train_ds))
    return image_batch, np.ones((len(image_batch), 1))

def generate_fake_datas(g_model, train_ds):
    image_batch, label_batch = next(iter(train_ds))
    fake_images = g_model.predict(image_batch)
    return fake_images, np.zeros( (len(fake_images), 1) )

def generate_discriminator_dataset(g_model, train_ds, batch_size):
    X_real, Y_real = generate_real_datas(train_ds)
    X_fake, Y_fake = generate_fake_datas(g_model, train_ds)
    X = np.concatenate(( X_real, X_fake ))
    Y = np.concatenate(( Y_real, Y_fake ))
    return X,Y

def generate_gan_dataset(train_ds):
    image_batch, label_batch = next(iter(train_ds))
    return image_batch, tf.one_hot(label_batch, 4)

In [8]:
def train_all_models(d_model, g_model, gan_model, adv_model, train_ds, num_epochs = 10, train_dis = 1):
    N_runs = len(train_ds)
    for i in range(num_epochs):
        print('Epoch : ', i)
        Dloss, Dacc, GD_loss, GAN_loss = 0,0,0,0
        for n in tq.tqdm( range(N_runs) ):
            
            X_dis, Y_dis = generate_discriminator_dataset(g_model, train_ds, batch_size)
            if(n > 0):
                dloss, dacc  = d_model.train_on_batch(X_dis, Y_dis)
            else:
                dloss, dacc = d_model.evaluate(X_dis, Y_dis, verbose = 0)
    
            X_gd, Y_gd   = generate_real_datas(train_ds)
            gloss        = gan_model.train_on_batch(X_gd, Y_gd)
            
            X_gan, Y_gan = generate_gan_dataset(train_ds)
            ganloss      = adv_model.train_on_batch(X_gan, Y_gan)
            
            
            Dloss = Dloss + dloss
            Dacc = Dacc + dacc
            GD_loss = GD_loss + gloss
            GAN_loss = GAN_loss + ganloss
            # Future Work : Joint Trainng Model
            #print('Discriminator Loss : ', round(dloss, 2), '  Accuracy', round(dacc,2),
              #'|| GAN Loss: ', round(gloss,2) , ' || Adv loss: ', round(ganloss,2))
        print('Discriminator Loss : ', round(Dloss/N_runs, 2), '  Accuracy', round(Dacc/N_runs,2),
              '|| GD Loss: ', round(GD_loss/N_runs,2) , ' || GAN loss: ', round(GAN_loss/N_runs,2))
        
        if( i%20 == 0 and train_dis == 1):
            g_model.save('saved_model/gen_'+ str(i) + '.h5')
            d_model.save('saved_model/dis_'+ str(i) + '.h5')

#  Execution 

In [9]:
sign_shape = (30, 30, 3)
num_classes = 43
num_epochs = 30

batch_size = 32
img_height = 30
img_width = 30
#images, labels = readTrafficSigns(data_location, 43)
#images = images/256

import pathlib
data_dir = pathlib.Path('Dataset/Signs/')
train_ds = tf.keras.preprocessing.image_dataset_from_directory( data_dir,  validation_split=0.2,
  subset="training",  seed=123,  image_size=(img_height, img_width),  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(  data_dir,  validation_split=0.2,
  subset="validation",  seed=123,  image_size=(img_height, img_width),  batch_size=batch_size)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Found 700 files belonging to 3 classes.
Using 560 files for training.
Found 700 files belonging to 3 classes.
Using 140 files for validation.


In [13]:
discriminator = get_discriminator(sign_shape)
generator = get_generator(sign_shape)
classifier = tf.keras.models.load_model(model_location +'model1.h5')

gan_model  = define_gan(generator, discriminator)
adv_model = define_adv(generator, classifier)

#encoder = Encoder(model_location)
#simulator = CarSimulator(model_location)

In [19]:
train_all_models(discriminator, generator, gan_model, adv_model, train_ds, num_epochs=40)

Epoch :  0



Discriminator Loss :  0.7   Accuracy 0.48 || GD Loss:  0.68  || GAN loss:  -3.94
Epoch :  1



Discriminator Loss :  0.7   Accuracy 0.46 || GD Loss:  0.68  || GAN loss:  -10.18
Epoch :  2



Discriminator Loss :  0.7   Accuracy 0.47 || GD Loss:  0.69  || GAN loss:  -10.53
Epoch :  3



Discriminator Loss :  0.7   Accuracy 0.49 || GD Loss:  0.69  || GAN loss:  -11.87
Epoch :  4


...


Discriminator Loss :  0.68   Accuracy 0.69 || GD Loss:  0.74  || GAN loss:  -15.98
Epoch :  36



Discriminator Loss :  0.68   Accuracy 0.68 || GD Loss:  0.73  || GAN loss:  -15.36
Epoch :  37



Discriminator Loss :  0.68   Accuracy 0.68 || GD Loss:  0.74  || GAN loss:  -15.18
Epoch :  38



Discriminator Loss :  0.67   Accuracy 0.68 || GD Loss:  0.74  || GAN loss:  -16.35
Epoch :  39



Discriminator Loss :  0.67   Accuracy 0.69 || GD Loss:  0.74  || GAN loss:  -15.69


In [14]:
generator = tf.keras.models.load_model('saved_model/gen_20.h5')

In [15]:
count = 0
total_count = 0
for x in tq.tqdm(train_ds):
    for y in range(len(x[0])):
        if( x[1][y] != 3):
            im = tf.expand_dims(x[0][y], 0)
            predb = generator(im )
            pred = classifier.predict(predb)
            
            total_count = total_count + 1
            count = count + (np.argmax(pred[0]) != x[1][y].numpy())
            #print(np.argmax(pred[0]) , x[1][y].numpy())
count, total_count

(354, 560)

In [16]:
count = 0
total_count = 0
for x in tq.tqdm(val_ds):
    for y in range(len(x[0])):
        if( x[1][y] != 3):
            im = tf.expand_dims(x[0][y], 0)
            predb = generator(im )
            pred = classifier.predict(predb)
            
            total_count = total_count + 1
            count = count + (np.argmax(pred[0]) != x[1][y].numpy())
            #print(np.argmax(pred[0]) , x[1][y].numpy())
count, total_count

(92, 140)

In [ ]:
            plt.subplot(1,2,1)
            plt.imshow(predb[0])
            plt.subplot(1,2,2)
            plt.imshow(im[0])